In [ ]:
# Clean the feeds
import pandas as pd
import codecs
input_file = codecs.open("parsed_feeds.csv", "r",encoding='utf-8', errors='replace')
output_file = open("parsed_df_clean.csv", "w")

def sanitize_characters(raw, clean):    
    for line in input_file:
        out = line
        output_file.write(line)
sanitize_characters(input_file, output_file)

In [ ]:
import pandas as pd
#load it in
floc = '/Users/Jay/AnacondaProjects/plutarch/get_episodes/preprocessed/'
fname = 'episode_df.csv'
#fname = 'parsed_df_clean.csv'
episode_df = pd.read_csv(floc+fname, encoding='utf-8')
episode_df.drop_duplicates(['description'],inplace=True)
episode_df=episode_df.drop(columns=['Unnamed: 0'])
# episode_df=episode_df.drop(columns=['Unnamed: 0.1'])

In [ ]:
len(episode_df)

In [ ]:
# Let's use a few regular expressions to clean up pour data, and save it back to disk 
# for future use

def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.replace(r"<p>", "")
    df[text_field] = df[text_field].str.replace(r" p ", "")
    df[text_field] = df[text_field].str.replace(r"\n", "")

    #df[text_field] = df[text_field].str.lower() 
    # commented out because Name Entity Recognition does not work with lower case text
    return df

episode_df = standardize_text(episode_df, "description")

floc = '/Users/Jay/AnacondaProjects/plutarch/get_episodes/preprocessed/'
fname = 'clean_df.csv'
episode_df=episode_df.dropna(axis=0, how='any')
episode_df=episode_df.reset_index(drop=True)
episode_df.to_csv(floc+fname)
clean_df.head()

In [ ]:
# Cleaning other aspects of the episode descriptions missed above:
for i in range(len(clean_df)):
    print('Fixing word ' + str(i) + ' of ' + str(len(clean_df)))
    clean_df['description'][i]=clean_df['description'][i].replace(r"br ", "")
    clean_df['description'][i]=clean_df['description'][i].replace(r"br ", "")
    clean_df['description'][i]=clean_df['description'][i].replace(r"nbsp", "")
    clean_df['description'][i]=clean_df['description'][i].replace(r"img", "")
    clean_df['description'][i]=clean_df['description'][i].replace(r'a href "target"_blank', "")
    clean_df['description'][i]=clean_df['description'][i].replace(r"href", "")
    clean_df['description'][i]=clean_df['description'][i].replace(r'"target"', "")
    clean_df['description'][i]=clean_df['description'][i].replace(r"_blank", "")
    clean_df['description'][i]=clean_df['description'][i].replace(r'"target"_blank', "")

In [ ]:
# Turns out some of my episode descriptions are pages long. Honestly, some people
# actually placed the entire episode transcript as the episode description. So, I found
# what looked nice on my site and truncated the max description length to 1200 characters. 
max_len=1200
for i in range(len(clean_df)):
    print('Truncating description ' + str(i) + ' of ' + str(len(clean_df)))
    if len(clean_df['description'][i]) > max_len: 
        this_string=clean_df['description'][i]
        this_string=this_string[:max_len] + '...'
        clean_df['description'][i] = this_string

In [2]:
# Load cleaned and processed data
import pandas as pd
floc = '/Users/Jay/AnacondaProjects/plutarch/flaskapp/dbs/'
fname = 'clean_df.csv'
clean_df = pd.read_csv(floc+fname)
clean_df=clean_df.drop(columns=['Unnamed: 0'])

clean_df.head()

,description,ep_title,pod_title
0,Her story is often told in a sort of sloppy sh...,"Peg Entwistle, Ghost of Hollywood",HowStuffWorks
1,Today we're revisiting an episodefrom Sarah an...,"SYMHC Classics: Mary Anning, Princess of Paleo...",HowStuffWorks
2,"In part two of this interview, Mindy busts som...",Interview: Mindy Johnson and the Women of Disn...,HowStuffWorks
3,Mindy Johnson has spent years tracking down th...,Interview: Mindy Johnson and the Women of Disn...,HowStuffWorks
4,Today we revisit a Sarah and Deblina episode f...,"SYMHC Classics: Victoria Woodhull, Little Quee...",HowStuffWorks


In [ ]:
floc = '/Users/Jay/AnacondaProjects/plutarch/flaskapp/dbs/'
fname = 'clean_df.csv'
clean_df.to_csv(floc+fname)
clean_df.head()

In [ ]:
import spacy

# nlp = spacy.load('/Users/Jay/AnacondaProjects/plutarch/glove_vec')
nlp=spacy.load('en_core_web_lg')
val=len(clean_df)
episode_vec=pd.DataFrame()
entity_list=['PERSON', 'NORP', 'ORG', 'GPE', 'LOC', 'EVENT']
for i in range(val):
    docs=nlp(clean_df['description'][i])
    print('Processing podcast episode ' + str(i) + ' of ' + str(val))
    #Get the text, NER, and vector for every word in every episode description
    for ent in docs.ents:
        #only pull in NER's of a specific category
        if ent.label_ in entity_list:
            episode_vec=episode_vec.append({'Word':ent.text,
                          'NER': ent.label_,
                          'Vectors':ent.vector,
                                            'Vec_norm':ent.vector_norm},
                                           ignore_index=True)
print('Done!')

In [ ]:
len(episode_vec)

In [ ]:
import pickle
# SAVE the word vectors as a pkl file
floc = '/Users/Jay/AnacondaProjects/plutarch/'
fname = 'episode_vec.pkl'
episode_vec.to_pickle(floc+fname)

In [ ]:
import pandas as pd
floc = '/Users/Jay/AnacondaProjects/plutarch/'
fname = 'episode_vec.pkl'
vec_df = pd.read_pickle(floc+fname)
vec_df.drop_duplicates(['Word'],inplace=True)
#episode_vec=episode_vec.drop(columns=['Unnamed: 0'])
vec_df = vec_df.dropna(axis=0, how='any')
vec_df = vec_df[~(vec_df[['Vec_norm']] == 0).any(axis=1)]
vec_df = vec_df.reset_index(drop=True)

In [ ]:
# Now that I've created a vocabulary list and associated embeddings, I need to create
# an index that details which episode(s) a given vocabulary word appears in
vec_indexed_df=pd.DataFrame()
# for i in range(len(vec_df)):
for i in range(len(vec_df)):
    episode_i=[]
    print('Processing word: '+ str(i) + ' of ' + str(len(vec_df)))
    for ii in range(len(clean_df)):
        if vec_df['Word'][i] in clean_df['description'][ii]:
            episode_i.append(ii)
    vec_indexed_df=vec_indexed_df.append({'ep_index':episode_i}, ignore_index=True)    

In [ ]:
floc = '/Users/Jay/AnacondaProjects/plutarch/'
fname = 'vec_indexed_df.pkl'
vec_indexed_df = pd.read_pickle(floc+fname)
vec_indexed_df=vec_final.reset_index(drop=True)

In [ ]:
vec_final=vec_indexed_df

In [ ]:
vec_final = pd.concat([vec_df.reset_index(drop=True),vec_indexed_df.reset_index(drop=True)], axis=1)

In [ ]:
vec_final

In [ ]:
# I should have done this earlier, but it's late and I'm tired. This cell removes
# every entry in vec_final that matches an author's name in clean_df

for i in range(len(vec_final)):
    print('Processing vocabulary word: '+ str(i) + ' of ' +str(len(vec_final)))
    for ii in range(len(clean_df)):
        if clean_df['pod_title'][ii] in vec_final['Word'][i]:
            vec_final=vec_final.drop([i])
            vec_final=vec_final.reset_index(drop=True)

In [ ]:
# SAVE the word vectors as a pkl file
floc = '/Users/Jay/AnacondaProjects/plutarch/'
fname = 'vec_final.pkl'
vec_final.to_pickle(floc+fname)

In [2]:
# LOAD the word vectors as a pkl file
import pandas as pd
floc = '/Users/Jay/AnacondaProjects/plutarch/'
fname = 'vec_final.pkl'
vec_final = pd.read_pickle(floc+fname)
vec_final=vec_final.reset_index(drop=True)
vec_final.head()


,NER,Vec_norm,Vectors,Word
0,GPE,6.820699,"[0.059734497, -0.2137025, 0.1405005, -0.344295...",Los Angeles
1,PERSON,5.471120,"[-0.1416, 0.351738, -0.027125001, 0.35031, 0.2...",Peg Entwistle
2,PERSON,6.203854,"[-0.0047858, -0.07088, -0.05422, 0.44179, -0.4...",Sarah
3,PERSON,3.262861,"[-0.0206945, -0.043768, 0.09138, -0.156215, 0....",Mary Anning
4,PERSON,6.698920,"[-0.41702, -0.013888, -0.12471, 0.22521, -0.07...",Mindy


In [ ]:
# Now that everything's loaded, I noticed that taking the time for an input to parse
# through the whole vocabulary to associate the input NER tag to each vocabulary tag
# really slows things down. This cell speeds up the time by breaking up my vec_final
# into multiple DB's based on the NER tag. 
import pandas as pd
PERSON=pd.DataFrame()
ORG_NORP=pd.DataFrame()
GPE_LOC=pd.DataFrame()
EVENT=pd.DataFrame()
for i in range(len(vec_final)):
    if vec_final['NER'][i]=='PERSON':
        PERSON=PERSON.append({'Word':vec_final['Word'][i],
                          'NER': vec_final['NER'][i],
                          'Vectors':vec_final['Vectors'][i],
                              'Vec_norm':vec_final['Vec_norm'][i],
                             'ep_index':vec_final['ep_index'][i]},
                            ignore_index=True)
    if vec_final['NER'][i]=='ORG' or vec_final['NER'][i]=='NORP':
        ORG_NORP=ORG_NORP.append({'Word':vec_final['Word'][i],
                          'NER': vec_final['NER'][i],
                          'Vectors':vec_final['Vectors'][i],
                              'Vec_norm':vec_final['Vec_norm'][i],
                                 'ep_index':vec_final['ep_index'][i]},
                            ignore_index=True)
    if vec_final['NER'][i]=='GPE' or vec_final['NER'][i]=='LOC':
        GPE_LOC=GPE_LOC.append({'Word':vec_final['Word'][i],
                          'NER': vec_final['NER'][i],
                          'Vectors':vec_final['Vectors'][i],
                              'Vec_norm':vec_final['Vec_norm'][i],
                               'ep_index':vec_final['ep_index'][i]},
                            ignore_index=True)
    if vec_final['NER'][i]=='EVENT':
        EVENT=EVENT.append({'Word':vec_final['Word'][i],
                          'NER': vec_final['NER'][i],
                          'Vectors':vec_final['Vectors'][i],
                              'Vec_norm':vec_final['Vec_norm'][i],
                           'ep_index':vec_final['ep_index'][i]},
                            ignore_index=True)
        

In [ ]:
PERSON.head()

In [ ]:
# Save each dataset to a pickle file
floc = '/Users/Jay/AnacondaProjects/plutarch/dbs/'
fname = 'PERSON.pkl'
PERSON.to_pickle(floc+fname)

floc = '/Users/Jay/AnacondaProjects/plutarch/dbs/'
fname = 'ORG_NORP.pkl'
ORG_NORP.to_pickle(floc+fname)

floc = '/Users/Jay/AnacondaProjects/plutarch/dbs/'
fname = 'GPE_LOC.pkl'
GPE_LOC.to_pickle(floc+fname)

floc = '/Users/Jay/AnacondaProjects/plutarch/dbs/'
fname = 'EVENT.pkl'
EVENT.to_pickle(floc+fname)

In [3]:
# This bit was used to act as the user input on the web app
import spacy
nlp = spacy.load('en_core_web_lg')
new_word='Winston Churchill'
doc=nlp(new_word)
for ent in doc.ents:
    print(ent.text, ent.label_,ent.vector, ent.vector_norm)
search_input=[ent.text,ent.label_,ent.vector, ent.vector_norm]

Winston Churchill PERSON [-1.63175002e-01  4.10860002e-01  6.16664998e-02 -3.87719989e-01
  5.55850044e-02 -1.82154998e-01 -1.17265001e-01 -2.57502496e-01
  3.00765008e-01  4.62919995e-02 -4.30979967e-01 -3.13439995e-01
 -2.34275013e-01 -2.54240006e-01  4.41834986e-01  1.02300003e-01
  2.44935006e-01 -9.74139050e-02  1.05328500e-01 -4.37175006e-01
 -3.90434980e-01 -2.28642493e-01  3.15421015e-01 -8.93349992e-04
  1.08522996e-01 -1.79177999e-01 -2.91974992e-01  3.56246501e-01
  3.68165016e-01  6.67055011e-01 -1.04500353e-03  2.01407492e-01
  1.26240000e-01  4.20819998e-01 -4.46404994e-01 -1.72386006e-01
  3.56975019e-01  2.27050006e-01 -3.94134998e-01 -2.01559991e-01
  4.96999979e-01  3.44145000e-01  1.93295494e-01 -2.97895014e-01
  8.49174976e-01  1.94920853e-01 -8.65084976e-02 -4.99580026e-01
  1.57440007e-01  6.58890009e-02 -6.43215001e-01  4.38335001e-01
  6.28965020e-01 -2.89594054e-01 -3.98349911e-02 -1.78284496e-01
  1.89311489e-01 -4.18009996e-01  1.87452003e-01  1.58311993e-01


In [4]:
# This code uses the NER tag generated from the input to load the dataset with the other
# relevant NER tags. 
import numpy as np
import pandas as pd
floc = '/Users/Jay/AnacondaProjects/plutarch/flaskapp/dbs/'

# numpy.dot(self.vector, other.vector) / (self.vector_norm * other.vector_norm)
dist_list=pd.DataFrame()
if search_input[1] == 'PERSON':
    fname = 'PERSON.pkl'
    vec_final = pd.read_pickle(floc+fname)
    print('Loaded ' + str(fname))
elif search_input[1]=='ORG' or search_input[1]=='NORP':
    fname = 'ORG_NORP.pkl'
    vec_final = pd.read_pickle(floc+fname)
    print('Loaded ' + str(fname))
elif search_input[1]=='GPE' or search_input[1]=='LOC':
    fname = 'GPE_LOC.pkl'
    vec_final = pd.read_pickle(floc+fname)
    print('Loaded ' + str(fname))
elif search_input[1]=='EVENT':
    fname = 'EVENT.pkl'
    vec_final = pd.read_pickle(floc+fname)
    print('Loaded ' + str(fname))

Loaded PERSON.pkl


In [5]:
# Compute the cosine similarity between the user input and every vocabulary word.
import numpy as np
dist_list=pd.DataFrame()
name_parts = list(map(lambda x: x.strip(), new_word.split()))
for ii in range(len(vec_final)):
    dist=np.dot(ent.vector, vec_final.Vectors[ii]) / (ent.vector_norm * vec_final.Vec_norm[ii])        
    print(vec_final['Word'][ii] + ', ' + str(dist))
    if dist < 1 and vec_final['Word'][ii] not in ent.text:
        cont = False
        # My word embeddings are highly sensitive to its own name. For example,
        # "Julius Caesar" returns many "Julius" and "Caesar" in my results. This
        # made sure the search input could not return any result with it's own name
        # but only for people. Half my EVENTS db starts with "The Battle of..."
        if search_input[1]=='PERSON':
            for name in name_parts:
                if name in vec_final['Word'][ii]:
                    cont = True
        if cont:
            continue
            
        dist_list=dist_list.append({'ep_index':vec_final['ep_index'][ii],
                                'Word':vec_final['Word'][ii],
                      'Distance':(dist)}, 
                     ignore_index=True)

Peg Entwistle, 0.20391922907944302
Sarah, 0.4155255087555446
Mary Anning, 0.41527647320086336
Mindy, 0.3224307565166877
Mindy Johnson, 0.4958724667199961
Victoria Woodhull, 0.4573047117681179
Magnus Hirschfeld, 0.3415045199830725
Anne Royall, 0.44788386121284
Theodore Roosevelt, 0.5822762830439476
Anne, 0.48585276852365467
Wilfrid, 0.2789959918112146
Ada Lovelace, 0.36430609733285313
Byron, 0.43356577643877686
John Humphrey Noyes, 0.5505297844810636
Christine de Pizan, 0.24447388405411488
Joan, 0.44663349375829076
Anne Byrn, 0.48585276852365467
John Herschel, 0.4715910790902846
Mary Robinette Kowal s, 0.3911167965818134
Tracy, 0.4094479111468412
Mary, 0.41527647320086336
Bessie Coleman, 0.4767345161637201
Bessie, 0.3620537010024382
Lucretia Mott, 0.3126679344669857
Quaker, 0.3856908330067966
Hedy Lamarr, 0.20055799732115526
Hedy, 0.17419945372935836
Hans, 0.3143413510710018
Mary Toft's, 0.3409999483823696
David Wyrick, 0.4566551107620303
John Quincy Adams, 0.5662195795407815
John Adams

Macario Garcia, 0.3685504300653212
Rinker Buck, 0.29029381064518167
Buck, 0.1637648558030636
Franz Liszt, 0.3558839261861721
Sybil Ludington, 0.3573062531929802
Jeanne de Clisson, 0.1786604436411599
Elizabeth Bathory, 0.4820593914364374
Dennis Carr, 0.5420461874102164
Emmy Noether, 0.30075025973966973
Violet Jessop, 0.2268189193880263
Major, 0.00670907119478681
Nathaniel Greene, 0.4915418372800598
Charles Cornwallis, 0.6029184786632882
Marion Carstairs, 0.3308231395435573
Joe, 0.3969592572623616
David E, 0.3089187542821491
Hoffman, 0.46727864206794634
Mary Ann Cotton, 0.39870195617167864
Jane, 0.4150261188760811
Cecil Rhodes, 0.45394527747522356
John Smith Moffat, 0.47385161361680894
Patricia Capone, 0.5085314206446702
Diana Loren, 0.44574138835601895
Tenderloin, 0.12092280770454174
Hokusai, 0.12595495507012236
Nate DiMeo, 0.2852141383114593
Nate, 0.2852141383114593
Charles IX, 0.494498468307348
Frederick Russel Burnham, 0.5719770309213176
Fritz Duquesne, 0.37978609956712334
Elizabeth 

Arthur Conan Doyle, 0.4214912323866891
Moriarty, 0.20526038247478842
Horace Wells, 0.3473902065311504
Amelia Earhart, 0.44895707798541257
Raymonde de Laroche, 0.13509201233519558
Harriet Quimby, 0.41373786605946705
Jacqueline Cochran, 0.46593355441947143
Amy Johnson, 0.5240067369854392
Ferdinand Demara, 0.3626913781537856
Lucas, 0.35480599706738447
Spielberg, 0.30436241824851135
Roy Chapman Andrews, 0.5273152805950372
Otto Rahn, 0.3482382021184715
Louise Boyd, 0.5221215537861017
Jim Reed's, 0.3712045622124086
Sam Starr, 0.4424137717124409
Sam, 0.3950754964348042
Dickens, 0.4531131558078371
Jesse James, 0.4685071245855935
Roger Casement, 0.3015148435168224
Frida Kahlo, 0.3044697401383332
Frida, 0.28915795834124675
Sultan Murad IV, 0.3181805053269176
Robert Browning's, 0.4193496830100854
Elizabeth Barrett's, 0.45711120121978804
Barrett, 0.3203424740197376
Lone Ranger, 0.1653062046993878
Reeves, 0.4648145768702681
Jack Johnson's, 0.3771187749171415
Tommy Burns, 0.39264302950438507
Johnson

Churchill, 0.9313221668549045
Pompeii, 0.23137807096425878
Charles Sumner, 0.6092014998089827
Sumner, 0.47148227021714423
Herod, 0.24548180060791094
King Herod, 0.3068586655767357
Jimmy Carter, 0.44996322800499633
Thach, 0.16143703835588769
de Maintenon, 0.024526938159860043
David Accords, 0.34420832147147523
John Snow's, 0.291608182972402
John Franklin, 0.5893623189309408
Eleanor, 0.4855420009757587
Mark Twain, 0.32167518018864394
Clyde Barrow, 0.436149310074129
Bonnie Parker, 0.48230770648885996
Jack Wilson, 0.4244956661354133
Sacco, 0.19658931373320224
Vanzetti, 0.1959473360241481
Hamilton, 0.6098804837375367
Jaws, 0.09345485623750578
Henry McCarty, 0.48967273174027054
Billy, 0.38470915737126954
Join Candace, 0.19570587561831287
Georgiana, 0.252518016706979
Rudolph II, 0.4208486789915902
Sarah Winchester, 0.5011999764986215
Teddy Roosevelt, 0.53744444901287
Marco Polo, 0.3597922655034018
Betsy Ross, 0.44256780721519223
Gandhi, 0.4166037753234147
Learn, 0.013498161007797818
Charlemag

Shirley Withers, 0.40600777678147054
Peter Shellard, 0.4518403141853949
Jonathan Luna, 0.430552369808971
Holly Wells, 0.2671273061759572
Jessica Chapman, 0.42533024621600324
Julian Buchwald, 0.4098320017409167
Carolynne Watson, 0.4044656134338165
Donna Wheeler, 0.37039165238674243
John, 0.469870067209634
Keith, 0.489021075666348
Colin, 0.47955287571977967
Hagley Woods, 0.29861558025221474
Lauria Bible, 0.2922123164957213
Ashley, 0.4344425750378181
Danny, 0.3715596635310306
Kathy Freeman, 0.4676574985436633
Freeman, 0.39247183749923764
Christine Sharrock, 0.4138960258311516
Marianne Schmidt, 0.38447437053565275
Wanda, 0.35923180221705564
Melvyn Bragg, 0.34450262581319063
Breslau Wroclaw, 0.22078853836319903
Bonhoeffer, 0.2645673819416302
Semitism, 0.22997833071588292
Stephen PlantDean, 0.4832392650543108
Simon Tillotson, 0.4480453865625525
Hobbes, 0.3283025433239595
Jacques de Vaucanson's, 0.18998918019170924
Simon SchafferProfessor of History, 0.31097219040547436
Bryn Mawr CollegeAnd, 

...

Rocio Gonzal, 0.17778572004574966
NARANJO, 0.16917747980670575
Paul Claudel Claudel, 0.4550485467357347
de Paul   Ys, 0.29958694298163807
Jean Lebrun   Non, 0.2795405628568787
Jean Louis Brunaux, 0.38236418966600083
Jean Louis BRUNAUX, 0.38236418966600083
La Gr, 0.14936690639267097
Nicolas Richer, 0.2857745923447062
Jean Lebrun   Les Grecs ne, 0.26455605816103167
Ainsi parlait, 0.10017681043601635
Jean Pierre Vernant quand il tenait, 0.32786952500725375
Hastie, 0.14994324431702266
Bruce George Peter Lee, 0.5498113466232679
Mo Blackwell, 0.44044195830490473
Theme Music, 0.06572070575335771
Quinn s Song, 0.2810877126332301
Sagar MBE, 0.3303017725468022
Colin Norris, 0.5077622940719855
Episode 46, -0.06310843562092078
Mohamed Malik, 0.31337741726787816
Heath Network, 0.23715032062323954
Sylvia Fleming, 0.5187149214439721
Sylvia Fleming's, 0.4726902041191624
Reeder, 0.28857821652864407
Ciaran Barnes, 0.422844874738554
John Gilligan, 0.4286393386137922
Emily O'Reilly, 0.41796603347709155
Ga

Yusuf Komunyakaa, 0.19856910787749982
Spongebob Squarepants, 0.2600688087314168
Patrick Swayze's, 0.34802214109565444
Shawn Sheehan, 0.40297909432169354
Tom Kapsidelis, 0.38414529662708174
Allison Quantz, 0.42749122536003686
Paula Vogel, 0.4592011258448075
Jacqueline Bixler, 0.3352504026990659
Bixler, 0.14382394801938558
Alex Parrish, 0.43278235416867167
Parrish, 0.3177272067670642
Kevin Hamed, 0.36622555649424776
James Vance, 0.4918503805385961
Gabrielle Smith, 0.42456430127743344
Dan Fagin, 0.33769355143753327
Kevin Bartram, 0.39967490927968463
Tim Seibles, 0.35372491011181745
Biz Beiderbecke, 0.123080418626708
Jacqueline Secoy, 0.41659687910641036
Sam Quinones, 0.36882465217856797
Noel Lobley, 0.32646407686271145
Joseph Roane, 0.4683527599599298
Margot Lee Shetterly, 0.5208285508681197
Peter Wallenstein, 0.42703355172064406
Mildred Loving, 0.33300938057058815
Katherine Johnson, 0.5631632890440468
Nadine Kabbani, 0.3385909942726024
Rick, 0.3568336167477332
George Crocker, 0.518900671

John Brodie, 0.48282911519685223
Poet Tim Siebles, 0.24696846519303683
Ann Rasmussen, 0.426062343661407
John Guthmiller, 0.469870067209634
Lisa Edwards Burrs, 0.4519914221936077
Reza Rafie, 0.3073674316109852
Wanda Johnson, 0.5345023104584818
Alan Levinovitz, 0.45721824564615454
Heather Griscom, 0.3377624488339427
Tom Marshall, 0.5014227947474318
Matthew Wolf, 0.3923746805306893
Pearl Chiu, 0.329098245531463
Glenn Strauss, 0.548411943863528
Bill Kissell, 0.1004846368522533
Donna Carty, 0.3448586402958973
Dena Radley, 0.33221443851902493
Sue Perdue, 0.24654351843709468
Kathleen Williams, 0.5269806697001608
Jack Beck, 0.3620479260397523
Wendy Welch, 0.4857361372914315
Eric Walden, 0.49121556212994905
Paul Kelly, 0.4634370003554819
Jay Sullivan, 0.48300571755466437
Anton Treuer, 0.3131002352492056
Victor Tan Chen, 0.404399025381325
Bernie Madoff, 0.3273607822756691
Andrew Cohen, 0.5158087331478498
Madoff, 0.24165448204859677
Nelson M  Walker, 0.4242551250584177
Amanda Herbert, 0.525628334

Marcel Ravidat, 0.3214285375206121
Yogi Berra, 0.26319645492829313
Ota, 0.1080386002564561
Eliezer Witztum, 0.18717662286895054
Moshe Kalian, 0.2287747169312604
Slate France, 0.28755593251829664
d "attachment_2041", -0.05098400356331582
800px, 0.08256382260120558
1000px, 0.06953067957075038
Sergey Larenkov, 0.23317303097925401
Harvey Levenstein, 0.5215110063892611
Andrew Buchanan, 0.5940428889598685
Bertram Gordon, 0.5936722821885965
Melanie Gordon Krob, 0.47186786073540016
Coco Chanel's, 0.283064356712875
Coco Chanel, 0.31330447947324097
Colin Field, 0.32684938917581585
Bar Hemingway, 0.3091414161893074
Pilar J, 0.3169873106824889
Alexis Ferenczi, 0.3613185689178761
Mary Blume, 0.4399028408891782
Frank Meier, 0.41150563521093947
Donald Kladstrup, 0.5065495534651934
Love Locks, 0.038042459471203535
eL Seed, 0.18373239563270777
Daniel Hourd, 0.4688100184787364
Page Views, -0.039831986109939155
Alice Hosched, 0.3775684151776962
Jean Pierre Hosched, 0.3643908788787719
Jacques Hosched, 0.3

Kindle br, 0.18351560749756385
Katie Horneshaw's, 0.2645693186482935
Megan, 0.3497042611440645
Ron Iddles, 0.37754078406376823
Jill Meagher s, 0.34872395773406184
Kindle Emily, 0.35646512256180696
Hilary Bonney, 0.43262141251401726
Kath Pettingill, 0.1399834022077362
Sallie Anne Huckstepp, 0.4518723726238811
Warren Lanfranchi, 0.5181374215343055
Roger Rogerson, 0.4015804269007556
Duncan McNab, 0.41778112388489635
Debra Krivoshow, 0.39543955490443705
Nanette Ellis, 0.3741584788251614
Nanette, 0.15081572613922145
Craig Ellis, 0.5147978536087857
Robert Farquharson, 0.546674083241589
Jai, 0.20447194065274785
Cindy Gambino, 0.39007530224221904
Marie Greening Zidan's, 0.3193494171955582
Janine Greening, 0.18202711884196332
673, -0.014260530805277625
638, -0.04484235401919129
637, -0.026592875469944113
615, -0.03037106332010653
602, -0.042827721889038335
555, -0.008585678667061189
Siebert, 0.2806791831265642
Les Gauchers Quintet, 0.09270114116475639
Shaw, 0.501636175543103
Ash Gray, 0.2048218

In [6]:
# sort the values in descending order and save the top 5 results (may change to 3 for 
# convenience and simplicity).
# NOTE: After completing this, I realise it would have been smarter to have only saved
# the top 3 = results for sake of memory.
result=dist_list.sort_values(by='Distance', ascending=False).head(3)
result=result.reset_index(drop=True)
result

,Distance,Word,ep_index
0,0.657164,Alexander Hamilton,"[147, 260, 916, 2042, 2043, 2081, 6212, 7547, ..."
1,0.656383,Elizabeth Schuyler Hamilton,[6212]
2,0.649598,Franklin Roosevelt,"[3274, 3290, 3346, 4306, 6433, 14006, 14092]"


In [14]:
# Combine podcast info from clean_df to display on my site. The 2 for-loops check to see
# how many episodes are indexed per word. If > 3, it returns a random 3. If < 3, it grabs
# whichever indices are listed.
import random
results_df=pd.DataFrame()
words=[]
for i in range(len(result)):
    words.append(result['Word'][i])
    if len(result['ep_index'][i]) >= 3:
        for ii in range(3):
            ep_list=random.choices(result['ep_index'][i], k=3)
            ep_i=ep_list[ii]
            print(clean_df['ep_title'][ep_i])
            results_df=results_df.append({'Description':clean_df['description'][ep_i],
                                     'ep_title':clean_df['ep_title'][ep_i],
                                      'Word':result['Word'][i],
                                     'pod_title':clean_df['pod_title'][ep_i]},
                                        ignore_index=True)
    else:
        for ii in range(len(result['ep_index'][i])):
            ep_i=ep_list[ii]
            print(clean_df['ep_title'][ep_i])
            results_df=results_df.append({'Description':clean_df['description'][ep_i],
                                     'ep_title':clean_df['ep_title'][ep_i],
                                      'Word':result['Word'][i],
                                     'pod_title':clean_df['pod_title'][ep_i]},
                                        ignore_index=True)

A Conspiracy Starring Aaron Burr
Elizabeth Schuyler Hamilton Born - August 9, 1757
Meet the Tailor
Meet the Tailor
제 709 편 - 명종의 시대, 막을 내리다
제 709 편 - 명종의 시대, 막을 내리다
Hard Cuddles with James Harding


In [15]:
results_df

,Description,Word,ep_title,pod_title
0,After Aaron Burr slew Alexander Hamilton in th...,Alexander Hamilton,A Conspiracy Starring Aaron Burr,HowStuffWorks
1,"Elizabeth Schuyler Hamilton, philanthropist an...",Alexander Hamilton,"Elizabeth Schuyler Hamilton Born - August 9, 1757",HowStuffWorks
2,The tailor 8217 s art is all in the cut Jour...,Alexander Hamilton,Meet the Tailor,Colonial Williamsburg History Podcasts - Image...
3,The tailor 8217 s art is all in the cut Jour...,Elizabeth Schuyler Hamilton,Meet the Tailor,Colonial Williamsburg History Podcasts - Image...
4,"709 ,",Franklin Roosevelt,"제 709 편 - 명종의 시대, 막을 내리다",KBS
5,"709 ,",Franklin Roosevelt,"제 709 편 - 명종의 시대, 막을 내리다",KBS
6,James Harding is putting as much energy and pa...,Franklin Roosevelt,Hard Cuddles with James Harding,Meshel Laurie


In [11]:
# Used this cell to ensure the site would call in and correctly place information on my site
for name in words:
    others=results_df[results_df['Word']==name]
    print('new section' + str(name))
    for index,row in others.iterrows():
        print('new_card')
        print(row['ep_title'])
        print('')
        print('')
        
        
    

new sectionAlexander Hamilton
new_card
PREVIEW - Worlds Apart


new_card
The Reynolds Pamphlet Live from NYCC Presents


new_card
Brown v The Board of Education


new sectionElizabeth Schuyler Hamilton
new_card
Meet the Tailor


new sectionFranklin Roosevelt
new_card
Episode 209-Dark Days for Chiang Kai-Shek


new_card
제 709 편 - 명종의 시대, 막을 내리다


new_card
Jeremi Suri, “Impossible Presidency: The Rise and Fall of America’s Highest Office” (Basic Books, 2017)




In [ ]:
##########################################################################################
#EVERYTHING BELOW THIS POINT IS ABOUT LOADING AND INSTALLING GLOVE AND GOOGLENEWS VECTORS#
##########################################################################################

In [ ]:
# To get the GloVe embeddings inserted into my model to try against spacy's provided
# word2vec model, I used this site: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/
# to generate the code below:

from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.300d.txt'
word2vec_output_file = 'word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

# After, I used spacy's prebuilt init method to load the new vectors as a spacy model
# Code to use in shell:
# python -m spacy init-model en /Users/Jay/AnacondaProjects/plutarch/glove_vec -v /Users/Jay/AnacondaProjects/histpod/word2vec.tx

In [ ]:
# Test to ensure the model is loading the GloVe embeddings properly
nlp_glove = spacy.load('/Users/Jay/AnacondaProjects/plutarch/glove_vec')
new_word=('Julius Caesar')
doc=nlp_glove(new_word)
for ent in doc.ents:
    print(ent.text, ent.label_,ent.vector, ent.vector_norm)
a=[ent.text,ent.label_,ent.vector, ent.vector_norm]

In [ ]:
############ Below is the processing for GoogleNews-vectors-negative300.bin.gz ############

In [ ]:
# To try out the GoogleNews embeddings, I tried the code below, which I took from this 
# SUPER useful site that made me feel infinitely better that others were having the same
# issue and even the guy who made spacy (Honnibol) was saying the loading isn't perfect for
#  spacy v2
# site: https://github.com/explosion/spaCy/issues/1533

import gensim
import spacy

# Path to google news vectors
gn_path = '/Users/Jay/AnacondaProjects/plutarch/GoogleNews-vectors-negative300.bin.gz'

# Load google news vecs in gensim
model = gensim.models.KeyedVectors.load_word2vec_format(gn_path, binary=True)

# Init blank english spacy nlp object
nlp = spacy.blank('en')

# Loop through range of all indexes, get words associated with each index.
# The words in the keys list will correspond to the order of the google embed matrix
keys = []
for idx in range(3000000):
    keys.append(model.index2word[idx])

# Set the vectors for our nlp object to the google news vectors
nlp.vocab.vectors = spacy.vocab.Vectors(data=model.syn0, keys=keys)

nlp.vocab.vectors.shape
(3000000, 300)

In [ ]:
nlp.to_disk('/Users/Jay/AnacondaProjects/plutarch/google_vec')

In [ ]:
import spacy
nlp_google = spacy.load('/Users/Jay/AnacondaProjects/plutarch/google_vec')

In [ ]:
##########################################################################################
#EVERYTHING BELOW THIS POINT IS ABOUT FGIURES#
##########################################################################################

In [ ]:
# This cell is used to create a histogram of all the Name Entity Recognition tag counts
import seaborn
ner_hist = seaborn.barplot(x=vec_final.NER.value_counts().index, y=vec_final.NER.value_counts())
fig = ner_hist.get_figure()
fig.savefig('ner_hist') 

In [ ]:
# Separating out the features
x = vec_final['Vectors'].tolist()

In [ ]:
x[0]

In [ ]:
from sklearn.preprocessing import StandardScaler
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pc_result = pca.fit_transform(x)

In [ ]:
import pandas as pd
groups = vec_final['NER'].tolist()
pc_plot=pd.DataFrame()
for i in range(len(pc_result)):
    pc_plot=pc_plot.append({'PC1':pc_result[i][0],
                            'PC2':pc_result[i][1],
                            'NER':groups[i]},
                          ignore_index=True)

In [ ]:
floc='/Users/Jay/AnacondaProjects/plutarch/plots/'
fname='pc_plot_db'
pc_plot.to_pickle(floc+fname)

In [ ]:
import pandas as pd
floc='/Users/Jay/AnacondaProjects/plutarchhistpod/plots/'
fname='pc_plot_db'
pc_plot=pd.read_pickle(floc+fname)

In [ ]:
EVENT_PC=pc_plot[pc_plot['NER']=='EVENT']
# GOP_PC=pc_plot[pc_plot['NER']=='GPE']
# frames = [GOP_PC,LOC_PC]
# GOP_LOC_PC=pd.concat(frames)
EVENT_PC

In [ ]:
>>> import seaborn as sns; sns.set(style="ticks", color_codes=True)
import pandas as pd
import numpy as np
np.random.seed(1974)

plot=sns.pairplot(x_vars=["PC1"], y_vars=["PC2"], data=pc_plot, hue="NER",palette="husl", height=6)
plot

In [ ]:
plot.savefig('NER_PC2.png')